# Invoking Q# callables from Python

The qsharp Python package makes it easy to call Q# operations and functions from within Python. Each Q# callable defined in `%%qsharp` magic cells, through calls to `qsharp.eval()`, or defined in a Q# project loaded via `qsharp.init()` is automatically added to the qsharp.code module in Python:

In [1]:
import qsharp

In [2]:
%%qsharp

import Std.Diagnostics.Fact;
operation TestBellPair() : Result[] {
    use qs = Qubit[2];
    H(qs[0]);
    CNOT(qs[0], qs[1]);
    let rs = MResetEachZ(qs);
    Fact(rs[0] == rs[1], "The qubits should be in the same state");
    rs
}

In [3]:
qsharp.code.TestBellPair()

[Zero, Zero]

Invoking these functions runs the corresponding Q# code against the sparse state simulator and returns the results as Python types, just as if they were invoked with a call to `qsharp.eval()`. They can also be imported from the module so they can be used like any other Python function:

In [4]:
from qsharp.code import TestBellPair

results = [TestBellPair() for _ in range(100)]
print(results)

[[Zero, Zero], [Zero, Zero], [One, One], [Zero, Zero], [Zero, Zero], [Zero, Zero], [Zero, Zero], [One, One], [Zero, Zero], [One, One], [Zero, Zero], [Zero, Zero], [Zero, Zero], [Zero, Zero], [Zero, Zero], [One, One], [One, One], [One, One], [Zero, Zero], [Zero, Zero], [Zero, Zero], [Zero, Zero], [One, One], [Zero, Zero], [Zero, Zero], [One, One], [Zero, Zero], [Zero, Zero], [Zero, Zero], [One, One], [Zero, Zero], [One, One], [Zero, Zero], [Zero, Zero], [One, One], [Zero, Zero], [One, One], [Zero, Zero], [One, One], [Zero, Zero], [Zero, Zero], [One, One], [One, One], [Zero, Zero], [Zero, Zero], [Zero, Zero], [Zero, Zero], [Zero, Zero], [Zero, Zero], [One, One], [Zero, Zero], [Zero, Zero], [Zero, Zero], [One, One], [Zero, Zero], [Zero, Zero], [One, One], [One, One], [Zero, Zero], [Zero, Zero], [One, One], [Zero, Zero], [Zero, Zero], [One, One], [Zero, Zero], [Zero, Zero], [One, One], [Zero, Zero], [Zero, Zero], [One, One], [One, One], [Zero, Zero], [Zero, Zero], [Zero, Zero], [One, One],

Arguments to Q# callables are converted from Python to the expected Q# type. If an argument cannot be converted to the right type, it will trigger a runtime exception:

In [5]:
qsharp.eval("""
    function AddTwoInts(a : Int, b : Int) : Int {
        return a + b;
    }
    """)

from qsharp.code import AddTwoInts

print(AddTwoInts(2, 3))

try:
    AddTwoInts(2, 3.0)
except TypeError as e:
    print(f"TypeError: {e}")

5
TypeError: 'float' object cannot be interpreted as an integer


If you define a Q# callables a namespace (or when loading from a Q# project), they will be exposed with a matching hiearchy of modules in Python:

In [6]:
%%qsharp

import Std.Diagnostics.DumpMachine;
namespace Foo {
    operation Bar() : Unit {
        use qs = Qubit[2];
        for q in qs {
            H(q);
        }
        DumpMachine();
        ResetAll(qs);
    }
}

In [7]:
from qsharp.code.Foo import Bar

Bar()

<table class="qs-stateTable">
  <style>
    .qs-stateTable thead tr {
      background-color: var(
        --vscode-list-hoverBackground,
        var(--jp-layout-color1, inherit)
      );
    }
    .qs-stateTable th {
      text-align: left;
      border: none;
    }
    .qs-stateTable tbody {
      pointer-events: none;
    }
    .qs-stateTable tbody td {
      text-align: left;
      border: none;
    }
    .qs-stateTable tbody td span {
      display: inline-block;
    }
    .qs-stateTable tbody tr:nth-child(even) {
      background-color: var(
        --vscode-list-hoverBackground,
        var(--jp-layout-color1, inherit)
      );
    }
  </style>
  <thead>
    <tr>
      <th>Basis State<br />(|𝜓₁…𝜓ₙ⟩)</th>
      <th>Amplitude</th>
      <th>Measurement Probability</th>
      <th colspan="2">Phase</th>
    </tr>
  </thead>
  <tbody>
    <tr>
  <td>
    <span>|00⟩</span>
  </td>
  <td>
    <span>0.5000+0.0000𝑖</span>
  </td>
  <td>
    <progress max="100" value="25.00000000000001"></progress>
    <span>25.0000%</span>
  </td>
  <td style="transform: rotate(0.0000rad)">↑</td>
  <td>
    <span>0.0000</span>
  </td>
</tr>
<tr>
  <td>
    <span>|01⟩</span>
  </td>
  <td>
    <span>0.5000+0.0000𝑖</span>
  </td>
  <td>
    <progress max="100" value="25.00000000000001"></progress>
    <span>25.0000%</span>
  </td>
  <td style="transform: rotate(0.0000rad)">↑</td>
  <td>
    <span>0.0000</span>
  </td>
</tr>
<tr>
  <td>
    <span>|10⟩</span>
  </td>
  <td>
    <span>0.5000+0.0000𝑖</span>
  </td>
  <td>
    <progress max="100" value="25.00000000000001"></progress>
    <span>25.0000%</span>
  </td>
  <td style="transform: rotate(0.0000rad)">↑</td>
  <td>
    <span>0.0000</span>
  </td>
</tr>
<tr>
  <td>
    <span>|11⟩</span>
  </td>
  <td>
    <span>0.5000+0.0000𝑖</span>
  </td>
  <td>
    <progress max="100" value="25.00000000000001"></progress>
    <span>25.0000%</span>
  </td>
  <td style="transform: rotate(0.0000rad)">↑</td>
  <td>
    <span>0.0000</span>
  </td>
</tr>

  </tbody>
</table>


$|\psi\rangle = \frac{1}{2}|00\rangle+\frac{1}{2}|01\rangle+\frac{1}{2}|10\rangle+\frac{1}{2}|11\rangle$

If you run `qsharp.init()`, the compiler and simulator state are reset and all functions exposed into Python are cleared:

In [8]:
qsharp.init()

try:
    Bar()
except qsharp.QSharpError as e:
    print(f"QsharpError: {e}")

QsharpError: callable not found
